# Exploratory Data Analysis and Basic Imputation of Missing Values

In [1]:
# import tensorflow as tf
# from tensorflow import keras

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
from sklearn import model_selection as ms
sklearn.set_config(print_changed_only=False)

import xgboost

import copy

import import_ipynb
from _Self_Written_Functions_Sheet import rmse

# print('The notebook is running with cuda:', tf.test.is_built_with_cuda())

importing Jupyter notebook from _Self_Written_Functions_Sheet.ipynb


# Data Exploration

In [2]:
# importing the datasets
hp_train = pd.read_csv('..\hp_2a_ranked_edited_train.csv', index_col=0)
hp_test = pd.read_csv('..\hp_2a_ranked_edited_test.csv', index_col=0)
hp_logsaleprice = pd.read_csv('..\hp_1a_no_imputation_logsaleprice.csv', index_col=0)
hp_saleprice = pd.read_csv('..\hp_1a_no_imputation_saleprice.csv')

# saving train index
hp_index = list(hp_train.index)

combo_nonedit = pd.DataFrame(np.concatenate([hp_train, hp_test]))
combo_nonedit.columns = hp_train.columns

for col in combo_nonedit.columns: 
    combo_nonedit[col] = combo_nonedit[col].astype('float', errors='ignore')

combo_cols = list(combo_nonedit.columns)

combo = copy.deepcopy(combo_nonedit)

In [3]:
combo

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,2-STORY 1946 & NEWER,RL,65.0,8450.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,4.0,3.0,PConc,4.0,3.0,1.0,6.0,706.0,1.0,0.0,150.0,856.0,GasA,5.0,1.0,5.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,8.0,7.0,0.0,None,Attchd,2003.0,2.0,2.0,548.0,3.0,3.0,3.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,2.0,2008.0,WD,Normal
1,1-STORY 1946 & NEWER ALL STYLES,RL,80.0,9600.0,1.0,0.0,Reg,Lvl,3.0,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6.0,8.0,1976.0,1976.0,Gable,CompShg,MetalSd,MetalSd,None,0.0,3.0,3.0,CBlock,4.0,3.0,4.0,5.0,978.0,1.0,0.0,284.0,1262.0,GasA,5.0,1.0,5.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,3.0,6.0,7.0,1.0,TA,Attchd,1976.0,2.0,2.0,460.0,3.0,3.0,3.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,5.0,2007.0,WD,Normal
2,2-STORY 1946 & NEWER,RL,68.0,11250.0,1.0,0.0,IR1,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2001.0,2002.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,4.0,3.0,PConc,4.0,3.0,2.0,6.0,486.0,1.0,0.0,434.0,920.0,GasA,5.0,1.0,5.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,6.0,7.0,1.0,TA,Attchd,2001.0,2.0,2.0,608.0,3.0,3.0,3.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,9.0,2008.0,WD,Normal
3,2-STORY 1945 & OLDER,RL,60.0,9550.0,1.0,0.0,IR1,Lvl,3.0,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7.0,5.0,1915.0,1970.0,Gable,CompShg,Wd Sdng,WdShing,None,0.0,3.0,3.0,BrkTil,3.0,4.0,1.0,5.0,216.0,1.0,0.0,540.0,756.0,GasA,4.0,1.0,5.0,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,4.0,7.0,7.0,1.0,Gd,Detchd,1998.0,1.0,3.0,642.0,3.0,3.0,3.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,0.0,None,0.0,2.0,2006.0,WD,Abnorml
4,2-STORY 1946 & NEWER,RL,84.0,14260.0,1.0,0.0,IR1,Lvl,3.0,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8.0,5.0,2000.0,2000.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,4.0,3.0,PConc,4.0,3.0,3.0,6.0,655.0,1.0,0.0,490.0,1145.0,GasA,5.0,1.0,5.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,4.0,9.0,7.0,1.0,TA,Attchd,2000.0,2.0,3.0,836.0,3.0,3.0,3.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,12.0,2008.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,2-STORY PUD - 1946 & NEWER,RM,21.0,1936.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,MeadowV,Norm,Norm,TwnhsI,2Story,4.0,7.0,1970.0,1970.0,Gable,CompShg,CemntBd,CemntBd,None,0.0,3.0,3.0,CBlock,3.0,3.0,1.0,1.0,0.0,1.0,0.0,546.0,546.0,GasA,4.0,1.0,5.0,546.0,546.0,0.0,1092.0,0.0,0.0,1.0,1.0,3.0,1.0,3.0,5.0,7.0,0.0,None,None,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,6.0,2006.0,WD,Normal
2913,2-STORY PUD - 1946 & NEWER,RM,21.0,1894.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4.0,5.0,1970.0,1970.0,Gable,CompShg,CemntBd,CemntBd,None,0.0,3.0,3.0,CBlock,3.0,3.0,1.0,3.0,252.0,1.0,0.0,294.0,546.0,GasA,3.0,1.0,5.0,546.0,546.0,0.0,1092.0,0.0,0.0,1.0,1.0,3.0,1.0,3.0,6.0,7.0,0.0,None,CarPort,1970.0,1.0,1.0,286.0,3.0,3.0,3.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0

### Creating functions to use as combinations for trials for linear and tree models

In [4]:
# creating a dictionary of the neigborhoods and ranks based on median value
list_neigborhoods = list(combo['Neighborhood'].unique())

n_dict_median = {}

for n in list_neigborhoods:
    n_dict_median[n] = hp_saleprice.loc[hp_train[hp_train['Neighborhood']==n].index].median()[0]

n_dict_median_sort = dict(sorted(n_dict_median.items(), key=lambda item:item[1], reverse=True))

n_list_median_sort = list(n_dict_median_sort.keys())

n_dict_median_ranking = {}
for i, n in enumerate(n_list_median_sort):
    n_dict_median_ranking[n] = i + 1

In [5]:
# functions:

def conditions_edit(df):
    # combine Railroad Adjacent and Railroad Within 200 for both railroads and both condition1 and 2
    df.loc[df['Condition1'].str.contains('RRN', na=False), 'Condition1'] = 'RRN'
    df.loc[df['Condition1'].str.contains('RRA', na=False), 'Condition1'] = 'RRA'

    df.loc[df['Condition2'].str.contains('RRN', na=False), 'Condition2'] = 'RRN'
    df.loc[df['Condition2'].str.contains('RRA', na=False), 'Condition2'] = 'RRA'

def nbrhd_rank(df):
    # creating a column replacing the neighborhood name with rank
    df['NbMedianRank'] = df['Neighborhood'].replace(n_dict_median_ranking)

def drop_nbrhd(df):
    if 'Neighborhood' in df.columns:
        df.drop('Neighborhood', axis=1, inplace = True)

def totallivsf_add(df):
    # creating a column with total square footage
    df[['TotalLivSF']] = df['GrLivArea'] + df['BsmtFinSF1'] + df['BsmtFinSF2']

def drop_grlivarea(df):
    if 'GrLivArea' in df.columns:
        df.drop(['GrLivArea'], axis=1, inplace = True)

def bsmtfin_add(df):
    # adding the column for whether basement is finished
    df['BsmtFin'] = df['BsmtFinSF1'] + df['BsmtFinSF2']
    df.loc[df['BsmtFin'] > 0, 'BsmtFin'] = 1 

def drop_bsmtfinsf12(df):
    if set(['BsmtFinSF1', 'BsmtFinSF2']).issubset(df.columns):
        df.drop(['BsmtFinSF1', 'BsmtFinSF2'], axis=1, inplace = True)

def totalporchsf_add(df):
    # adding the total SF of porches
    df['TotalPorchSF'] = df['OpenPorchSF'] + df['EnclosedPorch'] +\
    df['3SsnPorch'] + df['ScreenPorch']

def drop_porches(df):
    if set(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']).issubset(df.columns):
        df.drop(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'], axis=1, inplace = True)

def yrsremodtosold_add(df):
    # calculating the time between remodeling and selling the house
    df['YrsRemodToSold'] = df['YrSold'] - df['YearRemodAdd']

def drop_yrsoldremod(df):
    if set(['YrSold', 'YearRemodAdd']).issubset(df.columns):
        df.drop(['YrSold', 'YearRemodAdd'], axis=1, inplace = True)

def fireplace_yes(df):
    # Replacing number of fireplaces with either yes (1) or no (0)
    df['Fireplace'] = [1 if x > 0 else 0 for x in df['Fireplaces']]

def drop_fireplaces(df):
    if 'Fireplaces' in df.columns:
        df.drop(['Fireplaces'], axis=1, inplace = True)

def pool_yes(df): 
    # Replacing pool area with either yes (1) or no (0)
    df['Pool'] = [1 if x > 0 else 0 for x in df['PoolArea']]

def drop_poolarea(df):
    if 'PoolArea' in df.columns:
        df.drop(['PoolArea'], axis=1, inplace = True)

def totalbaths_add(df):
    # adding the number of bathrooms
    df['TotalBaths'] = df['FullBath'] + df['HalfBath'] * 0.5

def drop_baths(df):
    if set(['FullBath', 'HalfBath']).issubset(df.columns):
        df.drop(['FullBath', 'HalfBath'], axis=1, inplace = True)
    
def totalbsmtbaths_add(df):
    # adding the number of bathrooms
    df['TotalBsmtBaths'] = df['BsmtFullBath'] + df['BsmtHalfBath'] * 0.5
    
def drop_bsmtbaths(df):
    if set(['BsmtFullBath', 'BsmtHalfBath']).issubset(df.columns):
        df.drop(['BsmtFullBath', 'BsmtHalfBath'], axis=1, inplace = True)

### Creating functions from the models to call them below

In [11]:
def mlr_model(x_trn, y_trn, x_tst, y_tst):
    lm = LinearRegression()
    # parameters to use
    fitintercept = [True, False]
    gparam_mlr_1 = {'fit_intercept': fitintercept}
    # setting parameters
    gs_mlr_1 = ms.GridSearchCV(lm, gparam_mlr_1, cv=n_folds, refit=True, n_jobs=-1,
                               scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_mlr_1.fit(x_trn, y_trn)
    mlr_model.test_rmse = rmse(gs_mlr_1, y_tst, x_tst)    
    
def lasso_model(x_trn, y_trn, x_tst, y_tst):
    lasso = Lasso(random_state=state, max_iter=1000000, selection='random', fit_intercept=True)
    # parameters to use
    alphas = [0.0001, 0.000112, 0.000124, 0.000136, 0.000148]
    gparam_lasso_1 = {'alpha': alphas}
    # setting parameters
    gs_lasso_1 = ms.GridSearchCV(lasso, gparam_lasso_1, cv=n_folds, refit=True, n_jobs=-1,
                                 scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_lasso_1.fit(x_trn, y_trn)
    lasso_model.test_rmse = rmse(gs_lasso_1, y_tst, x_tst)

# def ridge_model(x_trn, y_trn, x_tst, y_tst):
#     ridge = Ridge(random_state=state, max_iter=10000, fit_intercept=True)
#     # parameters to use
#     solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
#     fitintercept = [True, False]
#     gparam_ridge_1 = {'solver': solvers,
#                       'fit_intercept': fitintercept, 
#                       'random_state': [state]}
#     # setting parameters
#     gs_ridge_1 = ms.GridSearchCV(ridge, gparam_ridge_1, cv=n_folds, refit=True, n_jobs=-1,
#                                  scoring='neg_root_mean_squared_error', return_train_score=True)
#     gs_ridge_1.fit(x_trn, y_trn)
#     ridge_model.test_rmse = rmse(gs_ridge_1, y_tst, x_tst)
    
def enet_model(x_trn, y_trn, x_tst, y_tst):
    enet = ElasticNet(random_state=state, max_iter=10000000, 
                      fit_intercept=True, selection='random')
    # parameters to use
    alphas = [1e-4, 2.5e-4, 5e-4]
    l1ratio = [0.5, 0.7, 1]
    precomputes = [True, False]
    warmstart = [True, False]
    gparam_enet_1 = {'alpha': alphas,
                      'l1_ratio': l1ratio,
                      'precompute': precomputes, 
                      'warm_start': warmstart,
                      'random_state': [state]}
    # setting parameters
    gs_enet_1 = ms.GridSearchCV(enet, gparam_enet_1, cv=n_folds, refit=True, n_jobs=-1,
                                scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_enet_1.fit(x_trn, y_trn)
    enet_model.test_rmse = rmse(gs_enet_1, y_tst, x_tst)
    
def rfr_model(x_trn, y_trn, x_tst, y_tst):
    rfr = RandomForestRegressor()
    gparam_rfr = {}
    # setting parameters
    gs_rfr = ms.GridSearchCV(rfr, gparam_rfr, cv=n_folds, refit=True, n_jobs=-1,
                             scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_rfr.fit(x_trn, y_trn)
    rfr_model.test_rmse = rmse(gs_rfr, y_tst, x_tst)
    
def gbm_model(x_trn, y_trn, x_tst, y_tst):
    gbm = GradientBoostingRegressor()
    gparam_gbm = {}
    gs_gbm = GridSearchCV(gbm, gparam_gbm, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_gbm.fit(x_trn, y_trn)
    gbm_model.test_rmse = rmse(gs_gbm, y_tst, x_tst)
    
def xgb_model(x_trn, y_trn, x_tst, y_tst):
    xgb_t = xgboost.XGBRegressor()
    gparam_xgb_t = {}
    gs_xgb_t = GridSearchCV(xgb_t, gparam_xgb_t, cv=n_folds, refit=True, n_jobs=-1,
                            scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_xgb_t.fit(x_trn, y_trn)
    xgb_model.test_rmse = rmse(gs_xgb_t, y_tst, x_tst)

In [7]:
def dum_scale():
    # dummify the variables
    hp_dum = pd.get_dummies(combo, drop_first=True)
    enc = OneHotEncoder()
    for feat in ['MoSold', 'YrSold']:
        if feat in hp_dum.columns:
            # encode the feature
            enc.fit(hp_dum[[feat]])
            # create array and then dataframe of the array
            oh_labels = enc.transform(hp_dum[[feat]]).toarray()
            mssubcl_dum = pd.DataFrame(oh_labels)
            mssubcl_dum.columns = enc.get_feature_names([feat])
            # Concatenate the dataframes and drop Id and original MSSubClass
            hp_dum = pd.concat([hp_dum, mssubcl_dum], axis=1, sort=False)
            hp_dum = hp_dum.drop([feat], axis=1)
    hp_dum_cols = list(hp_dum.columns)
    for col in hp_dum_cols:
        hp_dum[col] = MinMaxScaler().fit_transform(np.array(hp_dum[col]).reshape(-1,1))
        hp_dum.columns = hp_dum_cols
    dum_scale.hp_dum = copy.deepcopy(hp_dum)

In [12]:
# getting scores for all models using every possible combination of functions above

state = 0
counter = 0

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

# lists of the functions separated by whether something is changed or added
stuff_add = [nbrhd_rank, totallivsf_add, bsmtfin_add, totalporchsf_add, 
             yrsremodtosold_add, fireplace_yes, pool_yes, totalbaths_add,
             totalbsmtbaths_add, conditions_edit]

# list of functions that will drop columns
stuff_drop = [drop_nbrhd, drop_grlivarea, drop_bsmtfinsf12,  drop_porches,
              drop_yrsoldremod, drop_fireplaces, drop_poolarea,
              drop_baths, drop_bsmtbaths]

# list of models to use
models = [mlr_model, lasso_model,  enet_model, rfr_model, gbm_model, xgb_model]
# ridge_model,

subset_list = []

# numbers will be indices later down
funcs_scores = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}

import itertools

#https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements
#https://stackoverflow.com/questions/61313027/python-executing-all-permutations-of-list-of-functions

# getting all the combinations of a list for both add and drop
for r_a, r_b in zip(range(len(stuff_add)+1), range(len(stuff_add)+1)):
    com_list_a = list(itertools.combinations(stuff_add, r_a))
    com_list_b = list(itertools.combinations(stuff_drop, r_b))
    
    # getting all permutations of the 2 lists
    for subset in itertools.product(com_list_a, com_list_b):

        # reset the dataset to run the functions on a 'clean' dataset
        combo = copy.deepcopy(combo_nonedit)
        
        # adding the names of the functions we run thru to a list to track
        if subset == ((), ()):
            funcs_used_list = ['No changes to dataset']
        else:
            funcs_used_list = str(subset).split()[1::4]

        # if the list of functions is not in the list, run the functions
            # this is mostly to keep track of each permutation
        if funcs_used_list not in subset_list:

            # run the feat eng func
            for subs in subset:
                for s in subs:
                    s(combo)
#                     if callable(s) == True:
#                         s()
#                     else:
#                         continue

            # converting any numbers to numeric type
            for col in combo.columns: 
                combo[col] = combo[col].astype('float', errors='ignore')

            # run dummification and scaling
            dum_scale()

            # getting the train dataset for modeling
            hp = dum_scale.hp_dum.loc[hp_index]

            # getting the functions used to add to dictionary
            funcs_used = ', '.join(str(subset).split()[1::4])
            
            # making the dict key sensible
            if funcs_used == '())':
                funcs_used = 'No changes to dataset'

            # setting up train and test sets
            xtrain, xtest, ytrain, ytest = ms.train_test_split(hp, hp_logsaleprice,
                                                               test_size=0.2, random_state=state)
            # Must flatten to fit
            ytrain = ytrain.values.flatten()

            # run the models
            for i, model in enumerate(models):
                model(xtrain, ytrain, xtest, ytest)
                # add 1 for dictionaries w/in dictionary
                i+=1
                # add scores to dictionary
                funcs_scores[i][funcs_used] = model.test_rmse

            # add the list of functions already run to prevent re-runs
            subset_list.append(funcs_used_list)
            counter += 1
            print(f'Completed round {counter}')
        else:
            continue

Completed round 1
Completed round 2
Completed round 3
Completed round 4
Completed round 5
Completed round 6
Completed round 7
Completed round 8
Completed round 9
Completed round 10


ValueError: Columns must be same length as key

In [16]:
funcs_scores

{1: {'No changes to dataset': 0.11761582220844316,
  'nbrhd_rank, drop_nbrhd': 0.11687510094236864},
 2: {'No changes to dataset': 0.10372410349076429,
  'nbrhd_rank, drop_nbrhd': 0.10740118613289636},
 3: {'No changes to dataset': 0.10409966566052621},
 4: {'No changes to dataset': 0.12290651009174706},
 5: {'No changes to dataset': 0.11482935489647099},
 6: {'No changes to dataset': 0.1250608887296195}}

In [17]:
for k_1 in funcs_scores:
    k_min = min(funcs_scores[k_1], key=funcs_scores[k_1].get)
    v_min = funcs_scores[k_1][k_min]
    print(k_1)
    print(k_min)
    print(v_min) 
    print('#' * 50)

1
nbrhd_rank, drop_nbrhd
0.11687510094236864
##################################################
2
No changes to dataset
0.10372410349076429
##################################################
3
No changes to dataset
0.10409966566052621
##################################################
4
No changes to dataset
0.12290651009174706
##################################################
5
No changes to dataset
0.11482935489647099
##################################################
6
No changes to dataset
0.1250608887296195
##################################################


In [36]:
funcs_scores

{1: {'No changes to dataset': 0.11761582220844316,
  'nbrhd_rank, drop_nbrhd': 0.11687510094236864,
  'nbrhd_rank, drop_grlivarea': 0.11761582220844413},
 2: {'No changes to dataset': 0.10374124051679229,
  'nbrhd_rank, drop_nbrhd': 0.10721030527532438,
  'nbrhd_rank, drop_grlivarea': 0.10408495485040527},
 3: {'No changes to dataset': 0.10409966566052621,
  'nbrhd_rank, drop_nbrhd': 0.10758634673999228,
  'nbrhd_rank, drop_grlivarea': 0.10439812267347472},
 4: {'No changes to dataset': 0.12435759082044932,
  'nbrhd_rank, drop_nbrhd': 0.12515080070040543,
  'nbrhd_rank, drop_grlivarea': 0.13229286766827406},
 5: {'No changes to dataset': 0.11334023555852867,
  'nbrhd_rank, drop_nbrhd': 0.1148984046183187,
  'nbrhd_rank, drop_grlivarea': 0.1171717519523562},
 6: {'No changes to dataset': 0.1250608887296195,
  'nbrhd_rank, drop_nbrhd': 0.12487345301507552}}

In [37]:
for k_1 in funcs_scores:
    k_min = min(funcs_scores[k_1], key=funcs_scores[k_1].get)
    v_min = funcs_scores[k_1][k_min]
    print(k_1)
    print(k_min)
    print(v_min) 
    print('#' * 50)

1
nbrhd_rank, drop_nbrhd
0.11687510094236864
##################################################
2
No changes to dataset
0.10374124051679229
##################################################
3
No changes to dataset
0.10409966566052621
##################################################
4
No changes to dataset
0.12435759082044932
##################################################
5
No changes to dataset
0.11334023555852867
##################################################
6
nbrhd_rank, drop_nbrhd
0.12487345301507552
##################################################


In [6]:
# saving the imputed AND dummified AND scaled datasets
hp_train_full.to_csv('..\hp_3a_train_tree.csv')
hp_test_full.to_csv('..\hp_3a_test_tree.csv')

# For Linear

In [8]:
# dummify the variables
hp_dum = pd.get_dummies(combo, drop_first=True)

# scaling the dataset
from sklearn.preprocessing import MinMaxScaler
import copy

hp_dum_cols = list(hp_dum.columns)
hp_full = copy.deepcopy(hp_dum)

for col in hp_dum_cols:
    hp_full[col] = MinMaxScaler().fit_transform(np.array(hp_full[col]).reshape(-1,1))
    
hp_train_full = hp_full.loc[hp_index]
hp_test_full = hp_full.loc[1458:]

# saving the imputed AND dummified AND scaled datasets
hp_train_full.to_csv('..\hp_3a_train_linear.csv')
hp_test_full.to_csv('..\hp_3a_test_linear.csv')

In [9]:
# # converting columns containing numbers to numeric type
# for col in combo.columns: 
#     combo[col] = combo[col].astype('float', errors='ignore')
    
# combo.info()